In [1]:
import statsmodels.api as sm
import numpy as np 
from statsmodels.tsa.statespace.tools import (
    constrain_stationary_univariate, unconstrain_stationary_univariate)

class TVRegression2(sm.tsa.statespace.MLEModel):
    def __init__(self, y_t, x_t):
        exog = np.c_[x_t]  # shaped nobs x 2

        super(TVRegression2, self).__init__(endog=y_t, exog=exog, k_states=1, initialization='diffuse')

        # Since the design matrix is time-varying, it must be
        # shaped k_endog x k_states x nobs
        # Notice that exog.T is shaped k_states x nobs, so we
        # just need to add a new first axis with shape 1
        self.ssm['design'] = exog.T[np.newaxis, :, :]  # shaped 1 x 2 x nobs
        self.ssm['selection'] = np.eye(self.k_states)
        self.ssm['transition'] = np.eye(self.k_states)
        self.k_exog = 1

        #Which parameters need to be positive?
        self.positive_parameters = slice(1, 3)

    @property
    def param_names(self):
        return ['intercept', 'var.e', 'var.x.coeff', 'rho1']

    @property
    def start_params(self):
        """
        Defines the starting values for the parameters
        The linear regression gives us reasonable starting values for the constant
        d and the variance of the epsilon error
        """
        exog = sm.add_constant(self.exog)
        res = sm.OLS(self.endog, exog).fit()
        params = np.r_[res.params[0], res.scale, 0.001, 0.1]
        return params

    def transform_params(self, unconstrained):
        """
        We constraint the last three parameters
        ('var.e', 'var.x.coeff', 'var.w.coeff') to be positive,
        because they are variances
        """
        constrained = unconstrained.copy()
        constrained[self.positive_parameters] = constrained[self.positive_parameters]**2
        constrained[3] = constrain_stationary_univariate(constrained[3:4])
#         constrained[4] = constrain_stationary_univariate(constrained[4:5])
        return constrained

    def untransform_params(self, constrained):
        """
        Need to unstransform all the parameters you transformed
        in the `transform_params` function
        """
        unconstrained = constrained.copy()
        unconstrained[self.positive_parameters] = unconstrained[self.positive_parameters]**0.5
        unconstrained[3] = unconstrain_stationary_univariate(constrained[3:4])
#         unconstrained[4] = unconstrain_stationary_univariate(constrained[4:5])
        return unconstrained

    def update(self, params, **kwargs):
        params = super(TVRegression2, self).update(params, **kwargs)

        self['obs_intercept', 0, 0] = params[0]
        self['obs_cov', 0, 0] = params[1]
        self['state_cov'] = np.diag(params[2:3])
        self['transition', 0, 0] = params[3]
#         self['transition', 1, 1] = params[4]